In [1]:
import numpy as np
import time

## Definition 4

In [2]:
def subword_subset(a,n):
    
    split_list = []
    len_a = len(a)
    for i in range(len_a):
        delim = len_a - i
        if delim < n:
            break
        split_list.append(a[i:i+n]) 
    return split_list

def Omega(arr):

#     t0 = time.time()
    
    subword_set = []
    for i in range(1, len(arr)):
        subword_set.extend(subword_subset(arr, i))
    subword_set.append(arr)
    
#     t1 = time.time()
#     print("Total time elapsed: ", t1-t0)
    
    return subword_set

for i in range(10):
    Omega('001010')

In [3]:
Omega1 = Omega('001010')
print(Omega1)
Omega2 = Omega('002020')

['0', '0', '1', '0', '1', '0', '00', '01', '10', '01', '10', '001', '010', '101', '010', '0010', '0101', '1010', '00101', '01010', '001010']


## Previous implementation of Defn 4 
##### Note the difference in efficiency 

In [4]:
def subword_len(a,n=1):
    
    split_list = [a[i:i+n] for i in range(0, len(a), 1)]
    
    return [x  for x in split_list if len(x) == n and len(x)>0]

def subword(a):
    #t0 = time.time()
    try:
        b=[a]
        nmax=len(a)
        for n in range(nmax):
            c=subword_len(a,n)
            if len(c)>0:
                b=np.append(b,c)
        return b

    except: 
        return b

    #t1 = time.time()
    #print("Total time elapsed: ", t1-t0)

subword('34')
# for i in range(10):
#     print(subword('001010'))

array(['34', '3', '4'], dtype='|S2')

## Definition 5 - Jacard Similarity

##### Same as prior implementation as per ishanuc

In [5]:
def get_unweightedJac(S1,S2):
    return (len(set(S1).union(S2))-len(set(S1).intersection(S2)))/(0.0+len(set(S1).union(S2)))

# Definition 6 - Weighted Jacard Sequential Distance

In [6]:
# need to take the sum of the weighting
# Once this is finished, you can test individual parts then move onto putting it all together
# def sum_weight():
    
    


In [7]:
def get_weightedJac(S1,S2, sum_weight):
    sum_weight_union = sum_weight(list(set(S1).union(S2)))
    return (sum_weight_union - sum_weight(list(set(S1).intersection(S2))))/(0.0+sum_weight_union)

# Generate rando data

In [8]:
# check out MP for jacard
# mess with weighting

# Quasi Metric

In [9]:
# p0_assign

# Args:
# prob_vec: np array of our population disease prevalence probabilities

# Returns:
# prob_dict: dictionary whose keys are disease codes and values are
#            the associated probabilities

# assigns population probabilities to disease code
def p0_assign(prob_vec):
    prob_dict = {}
    for i in range(prob_vec.shape[0]):
        prob_dict[i] = prob_vec[i]
    return prob_dict


In [10]:
# hist_probs

# Args:
# histories: np array of patient histories
# prob_dict: dictionary whose keys are disease codes and values are
#            the associated probabilities

# Returns:
# hist_probs: dictionary whose key is the patient history index and
#             values are and the associated probabilities of each history

# assigns probabilities to histories
def hist_probs(histories, prob_dict):
    hist_probs = {}
    for i in range(histories.shape[0]):
        hist_probs[i] = prob_dict[histories[i][0]]
    return hist_probs


In [11]:
# qm_matrix_normalization

# Args:
# hist_probs: dictionary whose key is the patient history index and
#             values are and the associated probabilities of each history
# i: given row index 
# j: given column index

# Returns: the constant itself

# calculates the constant for the quasi matrix
def qm_matrix_normalization(hist_probs, i, j):
    return .5 * (((1.0/hist_probs[i]) + (1.0/hist_probs[j])) - 2)


In [12]:
# Args:
# histories: np array of patient histories
# hist_probs: dictionary whose key is the patient history index and
#             values are and the associated probabilities of each history

# Returns:
# qm_mat: our quasi-metric matrix

# creates our quasi-metric matrix - a probability adjusted jacard matrix in essence
# with unweighted Jacard matrix
def qm_Matrix(histories, hist_probs):
    length = len(hist_probs)
    qm_mat = np.zeros(shape = (length, length))
    for i in range(length):
        for j in range(length):
            qm_mat[i][j] = qm_matrix_normalization(hist_probs, i, j) * get_unweightedJac(subword(histories[i]), subword(histories[j]))
    return qm_mat


# Epsilon Neighborhood Step

In [25]:
# Args:
# qm_mat: our quasi-metric matrix
# epsilon: our selected epsilon value

# Returns:
# col_indexes: the indices, per column, of values in our quasi-matrix less than epsilon

# given an epsilon, figures out which indexes are in the epsilon neighborhood of the matrix
# basically for each row, we see which things in epsilon neighborhood then keep indexes,
# which we then use to figure out probabilities of occurrence
# qm matrix is gonna be symmetric, so we do calculations per column
def epsilon_indexes(qm_mat, epsilon):
    col_indexes = {}
    index_list = []
    for i in range(qm_mat.shape[0]):
        for j in range(qm_mat.shape[0]):
            if qm_mat[i][j] < epsilon:
                index_list.append(j)
        col_indexes[i] = index_list
        index_list = []
    return col_indexes


In [14]:
# updates probability of given history
# will cache rpt probabilities later
def hist_prob_update(histories, ep_ind, hist_probs, idx):
    #print ep_ind
    #print hist_probs
    pvecs = hist_probs
    for key in ep_ind:
        yesval = histories[key][idx]
        #print "This is our value we are comparing: ",yesval
        yes = 0 
        no = 0
        for i in range(len(ep_ind[key])):
            histval = histories[ep_ind[key][i]][idx]
            #print histval
            if yesval == histval:
                yes = yes + 1
            else:
                no = no + 1
            #print(yes, no)
        prob = yes/(yes * 1.0 + no)
        #print prob
        pvecs[key] = pvecs[key] * prob
    return pvecs
        
#hist_prob_update(dummy, test, hist_prob, 1)
# print House(dummy, p0, 3)
# print dummy

In [15]:
# is quasi metric, have to do bigger test : 

p0 = np.asarray([0.1, 0.05, 0.05, 0.2, 0.4, 0.2])
dummy = np.asarray([[2, 5, 5, 4, 3],[4, 4, 4, 1, 4], [4, 3, 5, 2, 3],[4, 2, 0, 0, 4],[4, 5, 0, 1, 0]])

# Putting it all together

In [23]:
def House(histories, p0, epsilon):
    p0 = p0_assign(p0)
    hist_prob = hist_probs(histories, p0)
    for i in range(1,len(histories[0])):
        qm = qm_Matrix(histories, hist_prob)
        #print(qm)
        epsilon_indices = epsilon_indexes(qm, epsilon)
        #print(epsilon_indices)
        hist_prob = hist_prob_update(histories, epsilon_indices, hist_prob, i)
        #print(hist_prob)
    return hist_prob


In [24]:
print House(dummy, p0, 3)

{0: 0.0125, 1: 0.1, 2: 0.04000000000000001, 3: 0.1, 4: 0.1}


In [ ]:
print dummy